In [75]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import os
import time
from sklearn import datasets
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score
from keras.datasets import cifar10
from keras.utils import to_categorical
import numpy as np

In [76]:
# load train and test dataset
def load_dataset():
	# load dataset
	(trainX, trainY), (testX, testY) = cifar10.load_data()
	# one hot encode target values
	#trainY = to_categorical(trainY)
	#testY = to_categorical(testY)
	return trainX, trainY, testX, testY

In [77]:
# scale pixels
def prep_pixels(train, test):
	# convert from integers to floats
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')
	# normalize to range 0-1
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0
	# return normalized images
	return train_norm, test_norm

In [78]:
trainX, trainY, testX, testY = load_dataset()
trainX, testX = prep_pixels(trainX, testX)

In [79]:
clf1 = tf.keras.models.load_model("cifar-vgg-baseline-holdout-100epochs_all_transfer.npy")

In [80]:
print(type(clf1))

<class 'keras.engine.sequential.Sequential'>


In [81]:
clf2 = tf.keras.models.load_model("cifar-mobileNet-baseline-holdout-100epochs_all_transfer.npy")

In [82]:
clf3 = tf.keras.models.load_model("cifar-resnet50-baseline-holdout-100epochs_all_transfer.npy")

In [83]:
class emsembleModel:
    def __init__(self, classifiers, numClasses):
        self.classifiers = classifiers
        self.numClasses = numClasses
        
    def predict_by_voting(self, dataX):
        preds = []
        for clf in self.classifiers:
            preds.append(clf.predict(dataX))
        
        resultPred = []
        M = len(self.classifiers)
        for i in range(len(dataX)):
            counting = [0 for _ in range(self.numClasses)]
            current_best_pred = preds[0][i]
            for j in range(M):
                pred = preds[j][i]
                predictedClass = np.argmax(pred)
                counting[predictedClass]+=1
                if counting[predictedClass] == max(counting):
                    current_best_pred = pred
            resultPred.append(current_best_pred)
        
        return resultPred
    
    def evaluate_by_voting(self, dataX, dataY):
        pred = self.predict_by_voting(dataX)
        ok = 0
        for i in range(len(pred)):
            expectedClass = dataY[i]
            predictedClass = np.argmax(pred[i])
            if predictedClass == expectedClass:
                ok += 1
        return ok/len(dataY)

    def predict_by_softmax(self, dataX):
        preds = []
        for clf in self.classifiers:
            preds.append(clf.predict(dataX))
        
        resultsPred = []
        M = len(self.classifiers)
        for i in range(len(dataX)):
            probs = []
            for j in range(M):
                pred = preds[j][i]
                probs.append(pred)
            compressed_probs = np.array(probs).sum(axis=0)
            compressed_probs = compressed_probs/M
            resultsPred.append(compressed_probs)
        
        return resultsPred
    
    def evaluate_by_softmax(self, dataX, dataY):
        pred = self.predict_by_softmax(dataX)
        ok = 0
        for i in range(len(pred)):
            expectedClass = dataY[i]
            predictedClass = np.argmax(pred[i])
            if predictedClass == expectedClass:
                ok += 1
        return ok/len(dataY)
    

In [84]:
classifiers = [clf1, clf2, clf3]

In [90]:
emsemble = emsembleModel(classifiers, 10)

In [86]:
acc = emsemble.evaluate_by_softmax(testX, testY)

313/313 [==============================] - 8s 21ms/step


In [87]:
print('Accuracy Emsemble using Softmax: %.3f' % (acc * 100.0))

Accuracy Emsemble using Softmax: 88.740


In [91]:
acc = emsemble.evaluate_by_voting(testX, testY)

313/313 [==============================] - 7s 21ms/step


In [92]:
print('Accuracy Emsemble using Voting: %.3f' % (acc * 100.0))

Accuracy Emsemble using Voting: 87.760
